# [4주차] 심화과제: 수능 국어 문제 GPT-4로 풀어보기

# [MY CODE] JSON Data 로드 및 확인

In [11]:
import pandas as pd

json_data = pd.read_json('./data/2023_11_KICE.json')
print(len(json_data))
json_data.head()

11


,id,paragraph,type,problems
0,2023_11_KICE_1-3,사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 ...,0,"[{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'..."
1,2023_11_KICE_4-9,(가)[A](중국에서 비롯된 유서(類書)는 고금의 서적에서 자료를 수집하고 항목별로...,0,"[{'question': '(가)와 (나)에 대한 설명으로 가장 적절한 것은?', ..."
2,2023_11_KICE_10-13,법령의 조문은 대개 ‘A에 해당하면 B를 해야 한다.’처럼 요건과효과로 구성된 조건...,0,"[{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'..."
3,2023_11_KICE_14-17,하루에 필요한 에너지의 양은 하루 동안의 총 열량 소모량인 대사량으로 구한다. 그중...,0,"[{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'..."
4,2023_11_KICE_18-21,혼례를 마친 후 최척이 아내와 함께 장모를 모시고 집으로 돌아오매 하인들이 기뻐했다...,1,"[{'question': '윗글에 대한 설명으로 가장 적절한 것은?', 'choic..."


In [12]:
json_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         11 non-null     object
 1   paragraph  11 non-null     object
 2   type       11 non-null     int64 
 3   problems   11 non-null     object
dtypes: int64(1), object(3)
memory usage: 484.0+ bytes


# [MY CODE] 클래스 활용 데이터 전처리

In [13]:
import json

class Problem:
    _number = 0

    def __init__(self, data, paragraph=None):
        if 'number' in data:
            self.number = data['number']
        else:
            Problem._number += 1
            self.number = Problem._number

        self.paragraph = data['paragraph'] if 'paragraph' in data else paragraph
        self.question_plus = data['question_plus'] if 'question_plus' in data else ''
        self.question = data['question']
        self.choices = data['choices']
        self.answer = data['answer']
        self.score = data['score']

    def predict(self, label):
        return self.score if int(label) == int(self.answer) else 0

    def to_json(self):
        problem_dict = {
            "number": self.number,
            "paragraph": self.paragraph,
            "question_plus": self.question_plus,
            "question": self.question,
            "choices": self.choices,
            "answer": self.answer,
            "score": self.score
        }

        return json.dumps(problem_dict, ensure_ascii=False, indent=4)

    def __str__(self):
        return self.question
    
class Paragraph:
    def __init__(self, data):
        self.id        = data['id']
        self.paragraph = data['paragraph']
        self.type      = data['type']
        self.problems  = []

        self._parse_problems(data['problems'])

    def _parse_problems(self, data_list):
        for data in data_list:
            problem = Problem(data, self.paragraph)
            self.problems.append(problem)

    def __str__(self):
        lines = f'Paragraph({self.id}): type({self.type})'
        for problem in self.problems:
            lines += f'\n\t- {problem.number}) {problem}'
        return lines

In [14]:
problem_list = []

for i in range(len(json_data)):
    paragraph_data = json_data.iloc[i]
    paragraph = Paragraph(paragraph_data)
    for problem in paragraph.problems:
        problem_list.append(problem)
    print(paragraph)

Paragraph(2023_11_KICE_1-3): type(0)
	- 1) 윗글의 내용과 일치하지 않는 것은?
	- 2) 다음은 학생이 독서 후 작성한 글의 일부이다. [A]를 바탕으로 ⓐ～ⓔ를 이해한 내용으로 가장 적절한 것은?
	- 3) 윗글을 읽고 ㉠에 대해 보인 반응으로 적절하지 않은 것은?
Paragraph(2023_11_KICE_4-9): type(0)
	- 4) (가)와 (나)에 대한 설명으로 가장 적절한 것은?
	- 5) [A]에 대한 이해로 적절하지 않은 것은?
	- 6) ㉮에 대한 이해를 바탕으로 ㉠, ㉡에 대해 파악한 내용으로 적절하지 않은 것은?
	- 7) ㉯를 반박하기 위한 ‘이수광’의 말로 가장 적절한 것은?
	- 8) (가), (나)를 읽은 학생이 <보기>의 임원경제지에 대해 보인 반응으로 적절하지 않은 것은?
	- 9) 문맥상 ⓐ～ⓔ와 바꾸어 쓰기에 적절하지 않은 것은?
Paragraph(2023_11_KICE_10-13): type(0)
	- 10) 윗글의 내용과 일치하지 않는 것은?
	- 11) ㉠에 대한 이해로 가장 적절한 것은?
	- 12) 윗글을 바탕으로 <보기>를 이해한 내용으로 가장 적절한 것은? [3점]
	- 13) 문맥상 ⓐ～ⓔ의 의미와 가장 가까운 것은?
Paragraph(2023_11_KICE_14-17): type(0)
	- 14) 윗글의 내용과 일치하지 않는 것은?
	- 15) 윗글을 읽고 추론한 내용으로 적절하지 않은 것은?
	- 16) ㉠, ㉡에 대한 이해로 가장 적절한 것은?
	- 17) 윗글을 바탕으로 <보기>를 탐구한 내용으로 가장 적절한 것은?
Paragraph(2023_11_KICE_18-21): type(1)
	- 18) 윗글에 대한 설명으로 가장 적절한 것은?
	- 19) 윗글의 인물에 대한 이해로 적절하지 않은 것은?
	- 20) ㉠～㉤에 대한 이해로 가장 적절한 것은?
	- 21) <보기>를 바탕으로 윗글을 감상한 내용으로 적절하지 않은것은?
Paragraph(2023_11_KICE_22

In [15]:
print(problem_list[0].to_json())

{
    "number": 1,
    "paragraph": "사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다. 최근 소통 공간이 온라인으로 확대되면서 독서를 통해 다른 독자들과 소통하며 즐거움을누리는 양상이 더 다양해지고 있다. 자신의 독서 경험을 담은 글이나 동영상을 생산 공유함으로써, 책을 읽지 않은 타인이 책과 소통하도록 돕는 것도 책을 통한 소통

# [MY CODE] OpenAI ChatClient

In [16]:
from openai import OpenAI
from os import environ
from dotenv import load_dotenv

load_dotenv()

models = ["gpt-4o-mini" , "gpt-4.1-mini"]

api_key = environ['OPENAI_API_KEY']

class OpenAIChatClient:
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)
        self.model = models[0]
        self.n = 1
        self.max_tokens = 4096
        self.temperature = 0.5
        self.frequency_penalty = 0.0

    def get_response(self, messages):

        if isinstance(messages, str):
            temp = [
                         {
                            "role": "user",
                            "content": messages
                        }
                    ]
            messages = temp

        response = self.client.chat.completions.create(
            model=self.model,
            n=self.n,
            max_tokens=self.max_tokens,
            temperature=self.temperature,
            frequency_penalty=self.frequency_penalty,
            messages=messages
        )
        return response.choices[0].message.content

In [17]:
chatgpt_client = OpenAIChatClient(api_key)
print(chatgpt_client.get_response('안녕 넌 누구니?'))

안녕하세요! 저는 AI 언어 모델입니다. 여러분의 질문에 답하고, 정보를 제공하며, 다양한 주제에 대해 대화할 수 있습니다. 무엇을 도와드릴까요?


# [MY CODE] 프롬프트

In [18]:
class PromptTemplate:
    def __init__(self, name, prompt):
        self.name = name
        self.prompt = prompt

    def compile(self, paragraph, question, question_plus, choice):
        return self.prompt.format(
            paragraph=paragraph,
            question=question,
            question_plus=question_plus,
            choice=choice
        )


prompt_templates = []

'''
TODO: 자유자재로 최대한 많은 종류의 프롬프팅 기법들을 활용하여 점수를 비교해보세요!!
'''

# COT (Chain of Thought)
prompt_templates.append(PromptTemplate('zero_shot_cot', """
다음은 지문, 질문, 그리고 선택지가 주어진 문제입니다.
먼저 질문을 이해하고, 단계별로 생각하여 가장 적절한 답을 선택해 주세요.

[지문]
{paragraph}

[질문]
{question}
{question_plus}

[선택지]
{choice}

이 문제를 해결하기 위해 단계별로 생각해봅시다.
1. 먼저 지문의 핵심 내용을 파악합니다.
2. 질문이 요구하는 것이 무엇인지 정확히 이해합니다.
3. 각 선택지를 지문과 연결하여 분석합니다.
4. 논리적 추론을 통해 가장 적절한 답을 선택합니다.

따라서 위 단계를 거쳐 생각한 결과, 가장 적절한 답은...

최종 답변은 번호만 입력해 주세요. 다른 설명이나 근거나 괄호 없이 오직 1단어로 숫자만 작성하세요. 예: 1 또는 2 또는 3 또는 4
- 올바른 예: 1
- 틀린 예: (1) 또는 "1번" 또는 "정답은 1입니다"
"""))

# 감정에 호소
prompt_templates.append(PromptTemplate('emotional', """
당신은 세계 최고의 문제 해결 전문가입니다. 수많은 학생들이 당신의 도움을 통해 학업적 성취를 이루었습니다.
지금 한 학생이 중요한 시험을 앞두고 있고, 이 문제의 정답을 찾는 것이 매우 중요합니다.
그 학생의 미래가 당신의 정확한 판단에 달려있습니다.

진심을 다해 다음 문제를 해결해 주세요:

[지문]
{paragraph}

[질문]
{question}
{question_plus}

[선택지]
{choice}

이 학생의 미래를 위해, 가장 정확한 답변을 제공해주세요. 당신의 풍부한 지식과 경험을 모두 동원하여 최선의 답을
선택해 주세요. 학생의 성공을 위해 신중하게 생각해 주세요.

최종 답변은 번호만 입력해 주세요. 다른 설명이나 근거나 괄호 없이 오직 1단어로 숫자만 작성하세요. 예: 1 또는 2 또는 3 또는 4
- 올바른 예: 1
- 틀린 예: (1) 또는 "1번" 또는 "정답은 1입니다"
"""))

# 원-샷 프롬프트
prompt_templates.append(PromptTemplate('one_shot', """
다음은 지문과 질문, 그리고 선택지가 주어진 문제입니다. 아래의 예시를 참고하여 문제를 풀어보세요.

[예시 문제]
[지문]
물은 생명 유지에 필수적인 요소이다. 인체의 약 60%는 물로 구성되어 있으며, 물은 체온 조절, 영양소 운반, 노폐물 제거 등 다양한 기능을 수행한다. 그러나 현대 사회에서는 많은 사람들이 충분한 양의 물을 섭취하지 않고 있다. 전문가들은 성인이 하루에 약 2리터의 물을 마셔야 한다고 권장하지만, 연구에 따르면 대부분의 사람들은 그보다 적은 양을 섭취한다.

[질문]
위 지문에서 강조하는 내용으로 가장 적절한 것은?

[선택지]
(1): 현대인들은 하루에 필요한 물의 양을 섭취하지 못하고 있다.
(2): 물은 인체 내에서 다양한 중요한 기능을 담당한다.
(3): 성인은 하루에 최소 2리터의 물을 마셔야 한다.
(4): 인체의 대부분은 물로 구성되어 있다.

[정답 도출 과정]
지문에서는 물이 생명 유지에 필수적이며 인체에서 다양한 기능을 한다는 점, 인체의 구성 비율, 권장 섭취량, 그리고 현대인들의 물 섭취 부족에 대해 언급하고 있습니다. 

지문의 핵심 주제는 물의 중요성과 현대인들의 불충분한 물 섭취입니다. 마지막 문장에서 "대부분의 사람들은 그보다 적은 양을 섭취한다"라고 강조하고 있으므로, 현대인들의 불충분한 물 섭취가 가장 중심이 되는 내용입니다.

정답: (1) 현대인들은 하루에 필요한 물의 양을 섭취하지 못하고 있다.

[실제 문제]
[지문]
{paragraph}

[질문]
{question}
{question_plus}

[선택지]
{choice}

[정답 도출 과정]

최종 답변은 번호만 입력해 주세요. 다른 설명이나 근거나 괄호 없이 오직 1단어로 숫자만 작성하세요. 예: 1 또는 2 또는 3 또는 4
- 올바른 예: 1
- 틀린 예: (1) 또는 "1번" 또는 "정답은 1입니다"
"""))

# 역할 부여 프롬프팅 (Role-based Prompting)
prompt_templates.append(PromptTemplate('role_based', """
당신은 세계적으로 유명한 문학 교수이자 비평가입니다. 하버드 대학에서 20년 이상 가르쳐 왔으며, 
텍스트 분석과 비판적 독해 능력이 탁월합니다. 당신은 학생들에게 텍스트를 깊이 이해하고 
분석하는 방법을 가르치는 데 전문가입니다.

다음 문제를 분석하고 해결해주세요:

[지문]
{paragraph}

[질문]
{question}
{question_plus}

[선택지]
{choice}

문학 교수로서의 전문성을 바탕으로, 지문을 철저히 분석하고 질문의 의도를 정확히 파악한 후
가장 타당한.선택지를 고르세요. 각 선택지의 근거를 지문에서 찾아보고, 텍스트 증거에 
기반한 판단을 내려주세요.

최종 답변은 번호만 입력해 주세요. 다른 설명이나 근거나 괄호 없이 오직 1단어로 숫자만 작성하세요. 예: 1 또는 2 또는 3 또는 4
- 올바른 예: 1
- 틀린 예: (1) 또는 "1번" 또는 "정답은 1입니다"
"""))

# 자기 일관성 프롬프팅 (Self-consistency)
prompt_templates.append(PromptTemplate('self_consistency', """
이 문제를 세 가지 다른 접근 방식으로 풀어보고, 가장 일관된 답을 선택하세요.

[지문]
{paragraph}

[질문]
{question}
{question_plus}

[선택지]
{choice}

접근법 1: 텍스트의 명시적 내용만 고려하여 분석
접근법 2: 텍스트의 함축적 의미와 논리적 추론을 통한 분석
접근법 3: 지문의 구조와 논지 전개 방식을 분석

각 접근법에 따른 분석 결과를 비교한 후, 최종 답을 선택하세요.
세 가지 접근법을 통해 가장 많이 지지되는 답이 정답일 가능성이 높습니다.

최종 답변은 번호만 입력해 주세요. 다른 설명이나 근거나 괄호 없이 오직 1단어로 숫자만 작성하세요. 예: 1 또는 2 또는 3 또는 4
- 올바른 예: 1
- 틀린 예: (1) 또는 "1번" 또는 "정답은 1입니다"
"""))

# 이유 열거 프롬프팅 (Reason + Enumerate)
prompt_templates.append(PromptTemplate('reason_enumerate', """
다음 문제를 풀기 위해 각 선택지의 타당성을 평가해보세요.

[지문]
{paragraph}

[질문]
{question}
{question_plus}

[선택지]
{choice}

각 선택지에 대해:
1. 해당 선택지가 옳다고 가정했을 때의 근거를 지문에서 찾아보세요.
2. 해당 선택지가 틀렸다고 반박할 근거도 지문에서 찾아보세요.
3. 근거의 강도를 1-10 척도로 평가하세요.


최종 답변은 번호만 입력해 주세요. 다른 설명이나 근거나 괄호 없이 오직 1단어로 숫자만 작성하세요. 예: 1 또는 2 또는 3 또는 4
- 올바른 예: 1
- 틀린 예: (1) 또는 "1번" 또는 "정답은 1입니다"
"""))
"""
다음 문제를 풀기 위해 각 선택지의 타당성을 평가해보세요.

[지문]
{paragraph}

[질문]
{question}
{question_plus}

[선택지]
{choice}

각 선택지에 대해:
1. 해당 선택지가 옳다고 가정했을 때의 근거를 지문에서 찾아보세요.
2. 해당 선택지가 틀렸다고 반박할 근거도 지문에서 찾아보세요.
3. 근거의 강도를 1-10 척도로 평가하세요.

예를 들어:
선택지 (1)
- 옳다고 보는 근거: (지문에서 해당 내용) / 강도: 7/10
- 틀리다고 보는 근거: (지문에서 해당 내용) / 강도: 3/10
결론: 선택지 (1)은 옳을 가능성이 높음

이런 방식으로 모든 선택지를 분석한 후, 가장 타당한 선택지를 답으로 선택하세요.

최종 답변은 번호만 입력해 주세요. 다른 설명이나 근거나 괄호 없이 오직 1단어로 숫자만 작성하세요. 예: 1 또는 2 또는 3 또는 4
- 올바른 예: 1
- 틀린 예: (1) 또는 "1번" 또는 "정답은 1입니다"
"""

# 디컴포지션 프롬프팅 (Decomposition)
prompt_templates.append(PromptTemplate('decomposition', """
이 문제를 작은 부분들로 나누어 체계적으로 접근해 봅시다.

[지문]
{paragraph}

[질문]
{question}
{question_plus}

[선택지]
{choice}

문제 분해:
1. 지문의 주요 주제는 무엇인가?
2. 지문에서 가장 중요한 사실이나 주장은 무엇인가?
3. 저자의 관점이나 태도는 어떠한가?
4. 질문이 요구하는 것은 정확히 무엇인가?

이제 각 선택지를 분석해 봅시다:
- 선택지 (1)은 위의 분석과 어떻게 연결되는가?
- 선택지 (2)는 위의 분석과 어떻게 연결되는가?
- 선택지 (3)은 위의 분석과 어떻게 연결되는가?
- 선택지 (4)는 위의 분석과 어떻게 연결되는가? (선택지가 더 있다면 계속)

이 분석을 바탕으로, 가장 적절한 답은...

최종 답변은 번호만 입력해 주세요. 다른 설명이나 근거나 괄호 없이 오직 1단어로 숫자만 작성하세요. 예: 1 또는 2 또는 3 또는 4
- 올바른 예: 1
- 틀린 예: (1) 또는 "1번" 또는 "정답은 1입니다"
"""))

# ANY_PROMPTING_METHOD (메타 프롬프팅)
prompt_templates.append(PromptTemplate('meta_prompting', """
당신은 텍스트 이해와 문제 해결에 특화된 AI 모델입니다. 다음 문제에 대해 최적의 답을 찾기 위한 
자체 분석 시스템을 가동하세요.

[지문]
{paragraph}

[질문]
{question}
{question_plus}

[선택지]
{choice}

[자체 지시사항]
1. 먼저 지문을 세 번 읽고 핵심 주제와 요점을 파악하세요.
2. 질문이 요구하는 것이 무엇인지 정확히 정의하세요.
3. 각 선택지에 대해 증거 기반 점수(1-10)를 매기세요.
4. 자신의 편향이나 직관보다는 텍스트 내 증거에 집중하세요.
5. 최종 결정을 내리기 전에 반대 입장에서 생각해보세요.
6. 가장 높은 점수를 받은 선택지를 답으로 선택하세요.

이 자체 분석 시스템을 통해 얻은 결론에 따라 답을 선택하세요.

최종 답변은 번호만 입력해 주세요. 다른 설명이나 근거나 괄호 없이 오직 1단어로 숫자만 작성하세요. 예: 1 또는 2 또는 3 또는 4
- 올바른 예: 1
- 틀린 예: (1) 또는 "1번" 또는 "정답은 1입니다"
"""))

# [MY CODE] 문제 풀기

In [19]:
def predict(json_text, prompt_template):
    problem = Problem(json.loads(json_text))

    choice_text = ''
    for idx, text in enumerate(problem.choices):
        choice_text += f'({idx+1}): {text}\n'

    prompt = prompt_template.compile(
        paragraph=problem.paragraph,
        question=problem.question,
        question_plus=problem.question_plus,
        choice=choice_text
    )

    #print(prompt_template.prompt)

    answer = chatgpt_client.get_response(prompt)
    score = problem.predict(answer)
    print(f'{problem.number}번 문제: (정답:{problem.answer}), (예측:{answer}) -> [점수:{score}점]' )
    return score

# [MY CODE] 프롬프트별 예측 및 비교

In [20]:
import os

# 기존 점수 데이터 로드
file_path = './data/score.csv'



# 파일 존재 여부 확인
if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
    # 파일이 존재하고 내용이 있는 경우
    score_df = pd.read_csv(file_path)
else:
    # 파일이 없거나 비어있는 경우 빈 데이터프레임 생성
    score_df = pd.DataFrame(columns=['name', 'score'])
    
    # data 디렉토리가 없으면 생성
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

print(score_df)

print(score_df)

for prompt_template in prompt_templates:
    total_score = 0

    if prompt_template.name in score_df['name'].values:
        # 이미 존재하면 점수 출력
        total_score = score_df.loc[score_df['name'] == prompt_template.name, 'score'].values[0]
    else:
        print(f"채점 시작({prompt_template.name})!")

        for problem in problem_list:
            score = predict(problem.to_json(), prompt_template)
            total_score += score

        # 존재하지 않으면 새로운 데이터프레임 추가
        new_row = {'name': prompt_template.name, 'score': total_score}
        score_df = pd.concat([score_df, pd.DataFrame([new_row])], ignore_index=True)
        score_df.to_csv(file_path, index=False)

    print(f"{prompt_template.name} 프롬프트 점수: {total_score}")

               name  score
0     zero_shot_cot     71
1         emotional     68
2          one_shot     67
3        role_based     65
4  self_consistency     64
               name  score
0     zero_shot_cot     71
1         emotional     68
2          one_shot     67
3        role_based     65
4  self_consistency     64
zero_shot_cot 프롬프트 점수: 71
emotional 프롬프트 점수: 68
one_shot 프롬프트 점수: 67
role_based 프롬프트 점수: 65
self_consistency 프롬프트 점수: 64
채점 시작(reason_enumerate)!
1번 문제: (정답:4), (예측:4) -> [점수:2점]
2번 문제: (정답:5), (예측:1) -> [점수:0점]
3번 문제: (정답:1), (예측:1) -> [점수:2점]
4번 문제: (정답:4), (예측:4) -> [점수:2점]
5번 문제: (정답:5), (예측:5) -> [점수:2점]
6번 문제: (정답:3), (예측:3) -> [점수:2점]
7번 문제: (정답:2), (예측:2) -> [점수:2점]
8번 문제: (정답:5), (예측:3) -> [점수:0점]
9번 문제: (정답:2), (예측:2) -> [점수:2점]
10번 문제: (정답:4), (예측:4) -> [점수:2점]
11번 문제: (정답:5), (예측:3) -> [점수:0점]
12번 문제: (정답:2), (예측:3) -> [점수:0점]
13번 문제: (정답:5), (예측:5) -> [점수:2점]
14번 문제: (정답:3), (예측:3) -> [점수:2점]
15번 문제: (정답:4), (예측:4) -> [점수:2점]
16번 문제: (정답:4), (예측:3) -> [점수

In [21]:
score_df

,name,score
0,zero_shot_cot,71
1,emotional,68
2,one_shot,67
3,role_based,65
4,self_consistency,64
5,reason_enumerate,64
6,decomposition,67
7,meta_prompting,58


프롬프트에 따라 구조화된 결과가 나오지 않아서 실제로는 데이터를 2번 처리해야하는 로직이 몇 개 있었음
실제로 데이터 포멧 자체에 문제가 있어서 다시 처리해야한다면

프롬프트 자체는 그대로 가져가고 데이터 포멧만 바꾸는 방식으로 처리하는게 가장 유지보수에 좋을 것 같음
[랭체인 버전](./deep.ipynb)